# Desarrollo del código necesario para la obtención de los datos principales.

Para este proyecto, los datos que utilizaremos principalmente serán aquellos que provienen de la API de CoinGecko. Utilizaremos la librería pycoingecko.

###  Trabajando con la librería pycoingecko.

Lo primero que debemos hacer, es entender los métodos que vamos a utilizar. A continuación, vamos a trabajar por primera vez con la librería.

In [13]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import timedelta,datetime

cg = CoinGeckoAPI()

In [14]:
# Función para crear dataframe de precios, market_cap y volumen de una criptomoneda específica, con datos de las últimas 24 horas, además de otro dataframe con datos de velas.
def df_onecoin_1day(coin:str,symbol,days=1):
    data = cg.get_coin_market_chart_by_id(
        id=coin,
        vs_currency = 'usd',
        days = days,
        localization = False
    )

    dfx = pd.DataFrame()
    j = 1
    for i in data.keys():
        if j == 1:
            dfx = pd.DataFrame(data[i],columns = ['Fecha',i])
        else:
            df = pd.DataFrame(data[i],columns = ['Fecha',i])
            dfx = dfx.merge(df)
        j += 1
    dfx.columns = ['Fecha','Precios','Market_Cap','Volumen']
    dfx['Fecha'] = pd.to_datetime(df.Fecha,unit='ms')
    
    for i in range(len(dfx)):
        if dfx.Fecha[i].minute <= 32 and dfx.Fecha[i].minute > 30:
            dfx.loc[i,'Fecha'] = pd.Timestamp(datetime.strptime(dfx.Fecha[i].strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M') 
                                                - timedelta(minutes=dfx.Fecha[i].minute - 30)
                                            )
        elif dfx.Fecha[i].minute >= 28 and dfx.Fecha[i].minute < 30:
            dfx.loc[i,'Fecha'] = pd.Timestamp(datetime.strptime(dfx.Fecha[i].strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M') 
                                                + timedelta(minutes=30 - dfx.Fecha[i].minute)
                                            )
        elif dfx.Fecha[i].minute <= 2 and dfx.Fecha[i].minute > 0:
            dfx.loc[i,'Fecha'] = pd.Timestamp(datetime.strptime(dfx.Fecha[i].strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M') 
                                                - timedelta(minutes=dfx.Fecha[i].minute)
                                            )
        elif dfx.Fecha[i].minute >= 58 and dfx.Fecha[i].minute <= 59:
            dfx.loc[i,'Fecha'] = pd.Timestamp(datetime.strptime(dfx.Fecha[i].strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M') 
                                                + timedelta(minutes=60 - dfx.Fecha[i].minute)
                                            )
        else:
            dfx.loc[i,'Fecha'] = pd.Timestamp(datetime.strptime(dfx.Fecha[i].strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M'))
    dfx = dfx[['Fecha','Market_Cap','Volumen']]
    
    df = pd.DataFrame(cg.get_coin_ohlc_by_id(
        id=coin,
        vs_currency = 'usd',
        days = days
        ),columns = ['Fecha','Abre','Alto','Bajo','Cierra'])
    df['Fecha'] = pd.to_datetime(df.Fecha,unit='ms')
    
    df = df.merge(dfx,how='left')
    
    df.to_csv(f'data/{symbol}_OHLC.csv',index=False)
    
    return df

def df_onecoin(coin:str,symbol,days=3764):
        data = cg.get_coin_market_chart_by_id(
                id = coin,
                vs_currency = 'usd',
                days = days,
                localization = False
                )
        dfx = pd.DataFrame()
        j = 1
        for i in data.keys():
                if j == 1:
                        dfx = pd.DataFrame(data[i],columns = ['Fecha',i])
                else:
                        df = pd.DataFrame(data[i],columns = ['Fecha',i])
                        dfx = dfx.merge(df)
                j += 1
        dfx.columns = ['Fecha','Precios','Market_Cap','Volumen']
        # dfx = dfx[['Fecha','Precios','Volumen']]
        dfx['Fecha'] = pd.to_datetime(dfx.Fecha,unit='ms')
        
        dfx.to_csv(f'data/{symbol}.csv',index=False)
        
        return dfx

In [9]:
# Aquí creamos una lista con los id's y otra con los símbolos.
coins = pd.DataFrame(cg.get_coins_markets(vs_currency ='usd')).head(15)['id'].to_list()
coins.remove('tether')
coins.remove('usd-coin')
coins.remove('staked-ether')
symbols = pd.DataFrame(cg.get_coins_markets(vs_currency ='usd')).head(15)['symbol'].to_list()
symbols.remove('usdt')
symbols.remove('usdc')
symbols.remove('steth')

# Posteriormente utilizamos las funciones para crear datasets del total de la vida de cada cripto y de un día atrás.
for i in range(len(coins)):
    # df_onecoin_1day(coin=coins[i],symbol=symbols[i])
    df_onecoin(coin=coins[i],symbol=symbols[i])

In [5]:
# Lista de criptomonedas que vamos a utilizar.
coins

['bitcoin',
 'ethereum',
 'binancecoin',
 'ripple',
 'cardano',
 'solana',
 'dogecoin',
 'tron',
 'polkadot',
 'matic-network',
 'shiba-inu',
 'litecoin']

In [6]:
symbols

['btc',
 'eth',
 'bnb',
 'xrp',
 'ada',
 'sol',
 'doge',
 'trx',
 'dot',
 'matic',
 'shib',
 'ltc']

In [11]:
pd.read_csv('data/doge_OHLC.csv').head()

,Fecha,Abre,Alto,Bajo,Cierra,Market_Cap,Volumen
0,2023-08-17 03:00:00,0.068046,0.068046,0.068046,0.068046,9.583335e+09,5.249407e+08
1,2023-08-17 03:30:00,0.067995,0.068179,0.067995,0.068179,9.575051e+09,5.720336e+08
2,2023-08-17 04:00:00,0.068254,0.068363,0.068123,0.068123,9.602263e+09,5.241845e+08
3,2023-08-17 04:30:00,0.068128,0.068129,0.068038,0.068050,9.567986e+09,5.369828e+08
4,2023-08-17 05:00:00,0.068015,0.068015,0.067919,0.067941,9.557298e+09,5.701636e+08


### Problemas con los datos de CoinGecko.

Los datos que nos entrega coingecko no son muy útiles puesto que nos da las temporalidades que quiere para los diferentes días atrás que extraemos, lo cual limita nuestros procedimientos.

Debido a esto, vamos a usar los datos más limpios que podamos obtener de esta API y los vamos a complementar con los de Binance (aunque si somos realistas, los datos que serán complementarios van a ser los de coingecko).